### Bert用于Token Classifcation的任务
Token Classifcation是对文本中的每个token（通常指的是经过分词后的词或者子词）分配一个标签。这类问题的典型应用包括：
1. 命名实体识别（NER, Named Entity Regcognition）：识别文本中的特定实体（人名、地名、机构等）
2. 词性标注（POS Tagging, Part-of-Speech Tagging）：为每个词分配一个语法类别，如名词、动词等
3. 分块标注（Chunking）：将句子中的词分组为具有特定语法功能的短语（名词短语、动词短语）<br>
### Token Classfication和Text Classification的区别
1. Token Classification是为输入文本中的每个词分配一个标签，通常用于序列标注任务；
2. Text Classification是为整个句子或者文本片段分配一个标签，通常用于情感分析、主题分类、垃圾邮件检测等任务。
### Fine-tuning步骤解析
fine-tuning指的是在预训练模型的基础上，使用特定任务的数据进一步训练模型，以便它能在该任务上表现更好。
1. 加载预训练模型
* 在自然语言处理任务中，通常会使用 Hugging Face 提供的预训练模型，如 BERT。这些模型已经在大量的文本数据上进行了预训练，具备丰富的语言理解能力。但为了让模型在特定任务（如命名实体识别）上表现良好，需要使用该任务的特定数据集进一步训练模型，即微调。
2. 数据预处理
* 对于 Token Classification 任务，每个输入文本的每个词语需要打上对应的标签。由于 BERT 模型的分词机制可能会将一个词拆成多个子词（subwords），因此需要确保标签与子词对齐。常见的做法是对一个词的所有子词使用相同的标签，或者仅对第一个子词打标签，其他子词忽略（标记为 -100，表示这些子词不参与损失计算）。
3. 设置训练参数
* 使用 TrainingArguments 设置训练的超参数，例如学习率、批次大小、训练周期数等。这些参数决定了训练的具体配置。
4. 评估指标
* 微调后，需要对模型的性能进行评估。对于命名实体识别任务，常用的评估指标有精确率（precision）、召回率（recall）、F1 值等。seqeval 是一个常用于序列标注任务的评估库。
5. 训练模型
* 使用 Hugging Face 的 Trainer API 来管理训练过程。Trainer 是一个高层封装，简化了训练循环、评估和保存模型的过程。
6. 评估模型
* 训练完成后，可以使用验证集或测试集对模型进行评估：

### 1. 导入库并加载数据

In [7]:
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

# 加载 WNUT 17 数据集
dataset = load_dataset("wnut_17")

# 查看标签的映射
label_list = dataset["train"].features["ner_tags"].feature.names

### 2. 加载分词器并进行数据处理

In [8]:
# 使用预训练的 BERT 分词器
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# 定义数据处理函数，确保输入和标签对齐
# 对输入的文本进行分词，并确保所有分词后的序列长度一致（truncation和padding），接着根据分词结果对标签进行对齐，确保每个子词有正确的标签
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding='max_length', is_split_into_words=True, max_length=128)
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # 忽略填充的 token
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # 第一个子词打标签
            else:
                label_ids.append(-100)  # 其他子词标记为 -100
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 对数据集进行映射处理
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 1287/1287 [00:00<00:00, 11961.42 examples/s]


### 3. 加载模型

In [9]:
# 加载BERT模型，进行Token Classification任务，并指定了输出层有多少个标签（命名实体的类别数）
model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4. 定义评估指标

In [10]:
# 使用 evaluate 库加载 seqeval
metric = evaluate.load("seqeval")

# 计算指标
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


### 5. 设置训练参数

In [11]:
# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5, # 学习率，控制模型参数更新的步长
    per_device_train_batch_size=16, # 每个设备上的训练批次大小为16
    per_device_eval_batch_size=16,
    num_train_epochs=3, # 模型对整个数据集训练3轮
    weight_decay=0.01,
)


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### 6. 初始化Trainer

In [12]:
# 使用 Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


### 7. 开始训练

In [13]:
# 开始训练
trainer.train()

# 进行评估
trainer.evaluate()


 33%|███▎      | 213/639 [01:53<04:14,  1.67it/s]

{'eval_loss': 0.2337796539068222, 'eval_precision': 0.6429906542056075, 'eval_recall': 0.41148325358851673, 'eval_f1': 0.5018234865061999, 'eval_accuracy': 0.9454013856225767, 'eval_runtime': 11.1752, 'eval_samples_per_second': 90.289, 'eval_steps_per_second': 5.727, 'epoch': 1.0}


                                                 
 67%|██████▋   | 426/639 [03:43<01:18,  2.73it/s]

{'eval_loss': 0.22604675590991974, 'eval_precision': 0.6454388984509466, 'eval_recall': 0.4485645933014354, 'eval_f1': 0.529287226534933, 'eval_accuracy': 0.9506769211212102, 'eval_runtime': 9.5739, 'eval_samples_per_second': 105.39, 'eval_steps_per_second': 6.685, 'epoch': 2.0}


 78%|███████▊  | 500/639 [04:18<01:05,  2.13it/s]

{'loss': 0.1578, 'grad_norm': 0.7395246028900146, 'learning_rate': 4.350547730829422e-06, 'epoch': 2.35}


                                                 
100%|██████████| 639/639 [05:37<00:00,  1.89it/s]


{'eval_loss': 0.24374958872795105, 'eval_precision': 0.6273584905660378, 'eval_recall': 0.4772727272727273, 'eval_f1': 0.5421195652173912, 'eval_accuracy': 0.9523294985063243, 'eval_runtime': 9.6408, 'eval_samples_per_second': 104.66, 'eval_steps_per_second': 6.638, 'epoch': 3.0}
{'train_runtime': 337.5295, 'train_samples_per_second': 30.166, 'train_steps_per_second': 1.893, 'train_loss': 0.13642226623630674, 'epoch': 3.0}


100%|██████████| 64/64 [00:09<00:00,  6.83it/s]


{'eval_loss': 0.24374958872795105,
 'eval_precision': 0.6273584905660378,
 'eval_recall': 0.4772727272727273,
 'eval_f1': 0.5421195652173912,
 'eval_accuracy': 0.9523294985063243,
 'eval_runtime': 9.5165,
 'eval_samples_per_second': 106.026,
 'eval_steps_per_second': 6.725,
 'epoch': 3.0}